<a href="https://colab.research.google.com/github/hanifbudiarto/rumus-kalibrasi/blob/main/kalibrasi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yaml, json, statistics
import pandas as pd

In [ ]:
output = "table"

In [ ]:
# === Fungsi interpolasi linear ===
def interpolasi_linear(x, data_titik, param):
    """
    Interpolasi linear untuk mencari koreksi berdasarkan titik acuan.
    """
    data_titik = sorted(data_titik, key=lambda t: t[param])
    for i in range(len(data_titik) - 1):
        x1, x2 = data_titik[i][param], data_titik[i + 1][param]
        y1, y2 = data_titik[i]["koreksi"], data_titik[i + 1]["koreksi"]
        if x1 <= x <= x2:
            koreksi = y1 + (y2 - y1) * (x - x1) / (x2 - x1)
            return round(koreksi, 4)
    return None  # jika di luar rentang

In [ ]:
# === Fungsi utama menerima data YAML (sudah dibaca) ===
def cari_koreksi(data_input, data_referensi, presisi=4):
    hasil = {}
    ref = data_referensi["referensi_kondisi"]

    for param, nilai in data_input["data_pengukuran"].items():
        terukur_awal = nilai["terukur_awal"]
        terukur_akhir = nilai["terukur_akhir"]

        koreksi_awal = interpolasi_linear(terukur_awal, ref[param]["titik_koreksi"], param)
        koreksi_akhir = interpolasi_linear(terukur_akhir, ref[param]["titik_koreksi"], param)

        terkoreksi_awal = terukur_awal + koreksi_awal
        terkoreksi_akhir = terukur_akhir + koreksi_akhir

        # Hitung rata-rata dari nilai terkoreksi
        rata_rata = (terkoreksi_awal + terkoreksi_akhir) / 2

        hasil[param] = {
            "awal": {
                "terukur" : round(terukur_awal, presisi),
                "koreksi" : round(koreksi_awal, presisi),
                "terkoreksi" : round(terkoreksi_awal, presisi)
            },
            "akhir": {
                "terukur" : round(terukur_akhir, presisi),
                "koreksi" : round(koreksi_akhir, presisi),
                "terkoreksi" : round(terkoreksi_akhir, presisi)
            },
            "rata_rata_terkoreksi": round(rata_rata, presisi),
            "standar_deviasi": round(statistics.stdev([terkoreksi_awal, terkoreksi_akhir]), presisi)
        }
    return hasil

In [ ]:
def tampilkan_tabel_hasil(hasil, presisi=2):
    """
    Menampilkan hasil perhitungan dalam bentuk tabel berwarna di Google Colab.

    Parameter:
      - hasil: dict hasil dari fungsi cari_koreksi()
      - presisi: jumlah angka di belakang koma (default = 2)
    """

    # === Susun data ke dalam list of dicts untuk DataFrame ===
    rows = []
    for param, data in hasil.items():
        rows.append({
            "Parameter": param.capitalize(),
            "Terukur Awal": data["awal"]["terukur"],
            "Terukur Akhir": data["akhir"]["terukur"],
            "Koreksi Awal": data["awal"]["koreksi"],
            "Koreksi Akhir": data["akhir"]["koreksi"],
            "Terkoreksi Awal": data["awal"]["terkoreksi"],
            "Terkoreksi Akhir": data["akhir"]["terkoreksi"],
            "Rata-rata Terkoreksi": data["rata_rata_terkoreksi"],
            "Standar Deviasi": data["standar_deviasi"],
        })

    df = pd.DataFrame(rows)

    # === Fungsi pewarnaan kolom ===
    def highlight_cols(x):
        style = pd.DataFrame('', index=x.index, columns=x.columns)
        style['Terukur Awal'] = 'background-color: #DAA520; color: white;'    # kuning tua
        style['Terukur Akhir'] = 'background-color: #DAA520; color: white;'
        style['Terkoreksi Awal'] = 'background-color: #8B0000; color: white;' # merah tua
        style['Terkoreksi Akhir'] = 'background-color: #8B0000; color: white;'
        return style

    # === Styling dan format angka ===
    styled_df = (
        df.style
        .apply(highlight_cols, axis=None)
        .format({
            "Terukur Awal": f"{{:.{presisi}f}}",
            "Terukur Akhir": f"{{:.{presisi}f}}",
            "Koreksi Awal": f"{{:.{presisi}f}}",
            "Koreksi Akhir": f"{{:.{presisi}f}}",
            "Terkoreksi Awal": f"{{:.{presisi}f}}",
            "Terkoreksi Akhir": f"{{:.{presisi}f}}",
            "Rata-rata Terkoreksi": f"{{:.{presisi}f}}",
            "Standar Deviasi": f"{{:.{presisi}f}}",
        })
    )

    display(styled_df)


In [ ]:
# === Bagian utama program ===
if __name__ == "__main__":
    # Baca dua file YAML
    with open("input_kondisi.yaml") as f1, open("referensi_kondisi.yaml") as f2:
        data_input = yaml.safe_load(f1)
        data_referensi = yaml.safe_load(f2)

    presisi = 2;

    # Pass ke fungsi
    hasil = cari_koreksi(data_input, data_referensi, presisi)

    # Cetak hasil
    if output == "json":
        print(json.dumps(hasil, indent=2))
    elif output == "table":
        tampilkan_tabel_hasil(hasil, presisi=presisi)


,Parameter,Terukur Awal,Terukur Akhir,Koreksi Awal,Koreksi Akhir,Terkoreksi Awal,Terkoreksi Akhir,Rata-rata Terkoreksi,Standar Deviasi
0,Suhu,25.50,25.40,-0.02,-0.02,25.48,25.38,25.43,0.07
1,Kelembapan,55.80,55.60,4.63,4.62,60.43,60.22,60.33,0.15
